In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [240]:
api_keys = json.load(open('api_keys.json', 'r'))

# 1. 주소 있는 데이터 읽어서 확인 

In [3]:
df = pd.read_csv('./jonmat2017.csv')
df = df[df.columns[:-1]]

In [4]:
#df = df[~df.주소.isnull()]
df[:3]

,총연번,의원번호,의원명,당,당ID,지역명,연월일,내역,지출액,사용처,분류,주소,의원지출액순위
0,17_000018,3,강길부,자유한국당,200,울산 울주군,2017.1.6,상임위원회의,"160,000",여의도소호정,간담회_식대,NaN,75
1,17_000019,3,강길부,자유한국당,200,울산 울주군,2017.1.7,지역현안회의,"84,000",두남일회,간담회_식대,NaN,75
2,17_000020,3,강길부,자유한국당,200,울산 울주군,2017.1.8,지역예산회의,"36,000",경기식당,간담회_식대,NaN,75


In [5]:
df['주소'] = df['주소'].apply(lambda x: x.strip() if type(x)==str else '')
df['지출액'] = df['지출액'].apply(lambda x: int(x.replace(',','')))

# 2. 주소 있는것 지오코딩

In [8]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_keys['google_maps'])

NameError: name 'api_keys' is not defined

In [13]:
def get_geocode(addr):
    geocode_result = gmaps.geocode(addr)
    if len(geocode_result) > 0:
        return geocode_result[0]['geometry']['location']
    else:
        return {'lat':0,'lng':0}

In [11]:
uniq = df['주소'].unique()

In [9]:
try:
    addr_dict = json.load(open('geocode_dict.json', 'r'))
except:
    addr_dict = {}

In [14]:
for addr in uniq:
    if addr in addr_dict:
        continue
    addr_dict[addr] = get_geocode(addr)

NameError: name 'gmaps' is not defined

In [45]:
json.dump(addr_dict, open('geocode_dict.json', 'w'))

In [17]:
addr_dict[''] = {'lat':0, 'lng':0}

In [15]:
for enum in enumerate(addr_dict):
    if enum[0] > 2:
        break
    print(enum)

(0, '서울특별시 영등포구 여의대방로68길 17')
(1, '서울특별시 영등포구 여의도동 45-20 동북빌딩 203-204')
(2, '서울특별시 강남구 개포로 623')


In [18]:
df_latlng = df['주소'].apply(lambda x: pd.Series({f:addr_dict[x][f] for f in ['lat','lng']}))
df_latlng[:3]

,lat,lng
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0


In [19]:
df = pd.concat([df, df_latlng], axis=1)

In [20]:
df.query("주소!=''")[:3].T

,345,346,347
총연번,17_001545,17_001554,17_001556
의원번호,5,5,5
의원명,강석호,강석호,강석호
당,자유한국당,자유한국당,자유한국당
당ID,200,200,200
지역명,경북 영양군영덕군봉화군울진군,경북 영양군영덕군봉화군울진군,경북 영양군영덕군봉화군울진군
연월일,2017.1.11,2017.1.17,2017.1.17
내역,지역현안관련정책개발보좌직원간담회,상임위법안소위관련보좌직원정책개발간담회,정책개발기자간담회
지출액,105000,160000,350000
사용처,또래오래치킨,청도,잔비어스


# - lat+lng 필드 만들기

In [40]:
df['latlng'] = df['lat'].astype(str) + '/' + df['lng'].astype(str)

# 3. 같은좌표의 다른 주소 표현
(도로명, 건물명, 띄어쓰기 등)

사용처와 좌표가 같은데 주소가 다른 것은 없는지(=2의 작업이 잘 되었는지 확인) 

In [41]:
def uniq_cnt(l):
    return len(l.unique())

In [48]:
addr_grouped = df.groupby(['사용처','latlng'])\
                .agg({'주소':[lambda x: len(x.unique()), min]})
addr_grouped[:3]

주소    
                   <lambda> min
사용처        latlng              
#INSHOP(샵  0.0/0.0        1    
(별관)목포세발낙지 0.0/0.0        1    
(사)노동자역사한내 0.0/0.0        1

In [49]:
addr_grouped.columns = ['uniq_cnt','addr']

In [47]:
addr_grouped = addr_grouped.query("uniq_cnt > 1 and addr != ''").reset_index()
addr_grouped[:3]

,사용처,lat,lng,uniq_cnt,addr


In [24]:
def change_addr(s):
    indices = df.query("사용처=='%s' and lat=='%s' and lng=='%s'"\
                      % (s['사용처'], s['lat'], s['lng'])).index
    df.loc[indices, '주소'] = s['addr']

In [25]:
addr_grouped.apply(change_addr, axis=1)
print()

# 같은 위치인데 다른사용처로 등록된 곳 뽑아보기

In [52]:
grouped = df.groupby(['latlng'])\
    .apply(lambda x: pd.Series({'uniq':len(x['주소'].unique()), 'sample':min(x['주소'])}))

In [53]:
grouped = grouped.query('uniq>1').reset_index()
grouped[:3]

,latlng,uniq,sample
0,0.0/0.0,2,
1,37.5011418/127.0020786,2,서울특별시 서초구 반포동 63-1
2,37.5183615/126.9338934,2,서울특별시 영등포구 국제금융로 112


In [82]:
df[df['latlng'].isin(grouped['latlng'][1:])].groupby(['사용처','latlng']).count()[['주소']].sort_values(by='latlng')[:6]

,,주소
사용처,latlng,
(주)현대그린푸드팔레스호텔,37.5011418/127.0020786,1
쉐라톤서울팔래스강남호텔,37.5011418/127.0020786,1
쉐라른서울팔래스강남호텔,37.5011418/127.0020786,1
서주산업개발(쉐라톤서울팔래스강,37.5011418/127.0020786,1
서주산업개발,37.5011418/127.0020786,49
다봉,37.5011418/127.0020786,4


In [60]:
import re

In [113]:
re.search(r'사업', '서주사업개발주식회사')

<_sre.SRE_Match object; span=(2, 4), match='사업'>

In [181]:
re.sub('호텔엔드|호텔앤도','호텔앤드','한화호텔앤도리조트')

'한화호텔앤드리조트'

In [232]:
def refine_typo(name):
    patterns = {
        '팔래스':'팔레스',
        '메리어트':'매리어트',
        '코엑스':'코액스',
        '호텔앤드':'호텔엔드|호텔앤도',
        '뚜레쥬르':'뚜레주르',
        '파리크라상':'파리크루아상',
        '':'\(주\)|\(주$',
        '참배나무골여의도점':'참나무골여의도점',
        '이즈미':'^이즈이',
        '전주이맛콩나물국밥':'전주참이맛콩나물국밥'
    }
    
    for p in patterns:
        name = re.sub(patterns[p], p, name)
        
    return name

In [233]:
def normalize_name(name):
    name = refine_typo(name)
    patterns = {
        '쉐라튼 팔래스 강남 호텔':[
            r'서주.*개발',
            r'팔래스.*강남',
            r'현대그린푸드팔래스호텔',
        ],
        '현대그린푸드국회의원식당':[
            r'현대그린푸드국회의',
        ],
        'JW메리어트서울':[
            r'JW메리어트서울',
            r'JW메리어트호텔서울',
        ],
    }
    
    for p in patterns:
        for reg in patterns[p]:
            if re.search(reg, name):
                return p
    return name

normalize_name('쉐라톤서울팔래스강남호텔')

'쉐라튼 팔래스 강남 호텔'

In [234]:
df_normalized = pd.concat([df['사용처'].apply(normalize_name), df[['latlng','총연번']]], axis=1)

In [235]:
df_normalized[df_normalized.latlng.isin(df_normalized.groupby(['latlng']).사용처.nunique().to_frame().query('사용처 > 1').index[1:])].groupby(['사용처','latlng']).count().sort_values(by='latlng')[90:]

,,총연번
사용처,latlng,
낙원,37.5296225/126.9203828,5
창고43(VIP점),37.5296225/126.9203828,1
창고43viP,37.5296225/126.9203828,1
창고43VIP점,37.5296225/126.9203828,1
창고43,37.5296225/126.9203828,1
KFC,37.5296225/126.9203828,1
버드나무식당,37.5296225/126.9203828,1
김,37.5297896/126.9192524,1
스시나고미,37.5297896/126.9192524,1


In [126]:
normalize_name('여의도소호정')

''

In [130]:
df[df['사용처'].apply(normalize_name) == '쉐라튼 팔래스 강남 호텔'][['사용처','주소']]

,사용처,주소
406,서주산업개발(쉐라톤서울팔래스강,서울특별시 서초구 반포동 63-1
1257,서주사업개발주식회사,
1775,서주사업개발,
2509,서주산업개발주식회사,
6489,다봉(서주산업개발주),
6814,서주산업개발,
6845,서주산업개발,
6860,서주산업개발,
6932,서주산업개발,
7048,서주산업개발,


# TODO: 같은 위치인데 다른사용처로 등록된 정리하기(어떻게?)

In [51]:
addr_grouped.query('uniq_cnt > 1')

,,uniq_cnt,addr
사용처,latlng,,
코레일유통,0.0/0.0,2,


# 기타(정리 안 함)

In [40]:
import requests
import json

In [90]:
df_all = pd.read_csv('jonmat2017.csv')
place_dict = {}

In [75]:
import requests
import json
import time

class NaverMaps():
    headers = {}
    url_format = "https://naveropenapi.apigw.ntruss.com/map-place/v1/"+\
        "search?query=%s&coordinate=126.929719,37.519749"
    
    def __init__(self, api_key_id, api_key):
        self.headers = {"X-NCP-APIGW-API-KEY-ID":api_key_id,
                        "X-NCP-APIGW-API-KEY":api_key,
                    }
    
    def search(self, query_string, retry=5):
        query_string = query_string.replace('#','')
        r = requests.get(self.url_format % query_string, headers=self.headers)
        d = json.loads(r.text)
        if d['status'] != 'OK':
            if retry > 0:
                print("%s retry: %s" % (place_name, retry))
                time.sleep(3)
                return self.search(query_string, retry=retry-1)
            else:
                raise BaseException(query_string, d)
        return d

In [77]:
m = NaverMaps(api_keys['naver_maps_id'], api_keys['naver_maps_key'])
r = m.search('비야게레로')
r

{'status': 'OK',
 'meta': {'totalCount': 1, 'count': 1},
 'places': [{'name': '비야게레로',
   'road_address': '서울특별시 강남구 봉은사로78길 12',
   'jibun_address': '서울특별시 강남구 삼성동 118-21 101호',
   'phone_number': '02-538-8915',
   'x': '127.0528107',
   'y': '37.5119717',
   'distance': 10883.506983308978,
   'sessionId': 'C8AqpmkBLhmWpuc1KvlZ'}],
 'errorMessage': ''}

In [119]:
for place_name in df['사용처'].unique():
    if place_name not in place_dict:
        #print("requested")
        r = m.search(place_name)
        place_dict[place_name] = r['places']
    #print(place_name, len(place_dict[place_name]))

(주)파리크라상천등산(평택방향) retry: 5
(주)파리크라상천등산(평택방향) retry: 4
친구찾기 retry: 5
친구찾기 retry: 4
우동전문점 retry: 5
신안홍탁 retry: 5
(주)부일씨앤디 retry: 5
옛삼덕수타면 retry: 5
가매야 retry: 5
차아나플레인 retry: 5
장수갈매기 retry: 5
해미집 retry: 5
올라6(주)호텔올라 retry: 5
올라6(주)호텔올라 retry: 4
주식회사공바위 retry: 5
고려정 retry: 5
한서외식산업주식회사(한우리한점) retry: 5
창고43시청점 retry: 5
광안리부산환집 retry: 5
0(1조 retry: 5
리틀타이 retry: 5
스타벅스(서여의도) retry: 5
(주)블랙스톤리조트양평 retry: 5
한화호텔앤드리조트(의원식당) retry: 5
행복의강 retry: 5
신씨화로신촌점 retry: 5
오늘은재영이네가 retry: 5
국회후생복지(커피숍) retry: 5
백리향(63빌딩) retry: 5
김박사손칼국수 retry: 5
돌산산장 retry: 5
삼송 retry: 5
금호횟집 retry: 5
호텔농심 retry: 5
호메로스호텔 retry: 5
해주냉면 retry: 5
주례한우갈비 retry: 5
어부횟집 retry: 5
향촌돼지국밥 retry: 5
여울돈단지 retry: 5
홍문 retry: 5
리안유통 retry: 5
구통성 retry: 5
노이프레소커피 retry: 5
마산식당 retry: 5
마산식당 retry: 4
한상바다 retry: 5
농업회사법인(주)대성축산물직판장 retry: 5
양평(하)휴게소탐앤탐스 retry: 5
땀뽕 retry: 5
제일면제소 retry: 5
자성화코다리 retry: 5
진사골순대국설렁탕 retry: 5
세프의국수전 retry: 5
월향조선횟집 retry: 5
한길식당 retry: 5
정읍(논산)휴게소 retry: 5
용인(강릉)휴게소 retry: 5
주식회사설풍 retry: 5
바다호텔 retry: 

In [131]:
json.dump(place_dict, open('./place_dict.json', 'w'))

In [120]:
len(place_dict)

6762

In [125]:
pd.Series([len(place_dict[place]) for place in place_dict]).value_counts()

5    2288
0    2201
1    1367
2     470
3     247
4     189
dtype: int64

In [134]:
pd.DataFrame(data={'road_address': '서울특별시 영등포구 국회대로 800 진미파라곤',
  'phone_number': '02-780-0363',
  'x': '126.9213120',
  'y': '37.5309780'}, index=[1])

,road_address,phone_number,x,y
1,서울특별시 영등포구 국회대로 800 진미파라곤,02-780-0363,126.9213120,37.5309780


In [198]:
def place_info(place_name, elem):
    d = place_dict[place_name]
    if len(d) != 1:
        return None
    if elem=='road_address' and d[0][elem]=='':
        elem='jibun_address'
    return d[0][elem]

In [199]:
x = []
columns = ['road_address','phone_number','x','y']
for c in columns:
    x.append(df['사용처'].apply(place_info, elem=c))

In [243]:
def place_series(place_name, return_all=False):
    d = place_dict[place_name]
    if len(d) == 0 or \
    ((not return_all) and len(d) > 1): # return that has only one match
        return pd.Series()
    
    addr_column = 'road_address'
    if d[0][addr_column]=='': # some places have no "road address"
        addr_column='jibun_address'
    return_dict = {elem:d[0][elem] for elem in ['phone_number','x','y']}
    return_dict['addr'] = d[0][addr_column]
    return pd.Series(return_dict)

In [245]:
naver_map_df = df['사용처'].apply(place_series)

In [258]:
df = pd.concat([df, naver_map_df], axis=1)

In [294]:
addr_minmax = df[df.notnull()['주소']].groupby(['의원명','사용처']).agg({'주소':[min, max]})
addr_minmax[:5]

주소                           
                                          min                        max
의원명 사용처                                                                 
강석호 (주)밥보                서욽특별시 영등포구 여의도동 13-3       서욽특별시 영등포구 여의도동 13-3
    (주)이랜드파크켄싱턴여의도       서울특별시 영등포구 여의도동 13-3       서울특별시 영등포구 여의도동 13-3
    (주)이와타쇼         서울특별시 영등포구 여의도동 36-2 지하1층  서울특별시 영등포구 여의도동 36-2 지하1층
    (주)제스코푸드청담지점             서울특별시 강남구 삼성동 88           서울특별시 강남구 삼성동 88
    (주)조선호텔호무랑              서울특별시 강남구 청담동 4-1          서울특별시 강남구 청담동 4-1

In [295]:
tmp_df = addr_minmax.droplevel(level=0, axis=1)

In [296]:
tmp_df[tmp_df['min'] != tmp_df['max']]

min  \
의원명 사용처                                               
김성수 (주)커피빈코리아                 경기도 시흥시 산천로 100번안길 16   
    물자리                         서울특별시 종로구 삼봉로 80-10   
    봉명성                            서울특별시 영등포구 74길 9   
    스시설                   경기도 고양시 일산동구 강송로 217번길 73   
    이꾸                        서울특별시 영등포구 국회대로 72길 6   
    정인면옥                     서울특별시 영등포구 국회대로 76길 22   
    진진                  서울특별시 영등포구 국회대로 72길 11 프린스텔   
    차이나플레인                   서울특별시 영등포구 국회대로 72길 11   
    쿠치나후                         서울특별시 영등포구 의사당대로 1   
    파리크라상                         서울특별시 영등포구 74길 12   
    파리크루아상                        서울특별시 영등포구 74길 12   
노웅래 군산오징어                      서울특별시 강남구 남부순환로 2615   
    양평신내해장국                    서울특별시 마포구 토정로 31길 24   
    지호마포점                         서울특별시 마포구 독막로 229   
    회관직원식당                      서울특별시 마포구 마포대로6길 10   
서청원 상도갈비타운                          서울시 동작구 상도동 522   
유승민 삼보정                        서울특별시 영등포구 여의도동 35-5   
    한화호텔앤리조트프레스센터점                서울특별시 중구 세종대로 124   
    향연                         서울특별시 영등포구 여의도동 23-1   
정우택 노조미             서울특별시 영등포구 여의도동 15-24 삼희익스콘벤처타워   
    차이나프로               서울특별시 영등포구 여의도동 LG트윈타워 동관5층   

                                                      max  
의원명 사용처                                                    
김성수 (주)커피빈코리아                       경기도 시흥시 신천로 100번안길 16  
    물자리                               서울특별시 종로구 상봉로 80-10  
    봉명성                             서울특별시 영등포구 국회대로 74길 9  
    스시설                          경기도 고양시 일산동구 강송로217번길 73  
    이꾸                               서울특별시 영등포구 국회대로72길 6  
    정인면옥                          서울특별시 영등포구 국회대로 76번길 22  
    진진                             서울특별시 영등포구 국회대로 76길 11  
    차이나플레인                           서울특별시 영등포구 여의도동 13-3  
    쿠치나후                            서울특별시 영등포구 의사당대로1 의정관  
    파리크라상                          서울특별시 영등포구 국회대로 74길 12  
    파리크루아상                         서울특별시 영등포구 국회대로 74길 12  
노웅래 군산오징어                          서울특별시 마포구 마포대로 14가길 18  
    양평신내해장국                           서울특별시 마포구 토정로31길 24  
    지호마포점                           서울특별시 영등포구 국회대로74길 19  
    회관직원식당                             서울특별시 영등포구 의사당대로 1  
서청원 상도갈비타운                              서울특별시 동작구 상도동 522  
유승민 삼보정                         서울특별시 영등포구 여의도동 35-5 종합상가  
    한화호텔앤리조트프레스센터점                       서울특별시 중구 세종대로124  
    향연                              서울특별시 종로구 사직로 10길 9-6  
정우택 노조미             서울특별시 영등포구 여의도동 15-24 삼희익스콘벤처타워 B113호  
    차이나프로                        서울특별시 영등포구 여의도동 맨하탄빌딩 1층

In [328]:
df.query("주소>'' and addr>''")

,총연번,의원번호,의원명,당,당ID,지역명,연월일,내역,지출액,사용처,분류,주소,의원지출액순위,Unnamed: 13,phone_number,x,y,addr
351,17_001581,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.7,지역현안관련의견청취기자간담회,"232,000",해우리서여의도점,언론_기자식대등,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
356,17_001591,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.13,전국위원회관련간담회,"180,600",해우리서여의도점,간담회_식대,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
360,17_001609,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.20,국정현안관련의견청취기자간담회,"160,000",스시나고미,언론_기자식대등,서울특별시 영등포구 국회대로 780,4,NaN,02-783-6409,126.9191480,37.5299082,서울특별시 영등포구 국회대로 780 엘지에클라트오피스텔 B105호
363,17_001619,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.22,정책개발기자간담회,"275,700",해우리서여의도점,언론_기자식대등,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
378,17_001680,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.3.29,지역주요현안및상임위관련등정책개발보좌직원간담회,"474,200",나리스키친,사무실_식대비,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
379,17_001681,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.3.29,지역주요현안및상임위관련등정책개발보좌직원간담회,"79,000",나리스키친,사무실_식대비,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
382,17_001692,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.6,정책개발기자간담회,"263,000",장어가조타,언론_기자식대등,서울특별시 영등포구 여의도동 36,4,NaN,02-780-6030,126.9264116,37.5201766,서울특별시 영등포구 의사당대로 127
384,17_001697,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.11,정책개발보좌직원간담회,"232,000",해우리서여의도점,사무실_식대비,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
387,17_001701,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.12,정책개발기자간담회,"313,410",나리스키친,언론_기자식대등,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
393,17_001717,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.18,정책개발보좌직원간담회(진주출장),"325,000",천황식당,사무실_식대비,경상남도 진주시 대안동 4-1,4,NaN,055-741-2646,128.0844789,35.1961301,경상남도 진주시 촉석로207번길 3
